# 實作PSO訓練類神經網路

經過ch1~5的介紹，相信要編寫一個粒子群演算法的優化器並不是難事。比較需要知道的是優化器如何在`trian_tool.train_batch`中運作的，而優化器需要符合什麼條件？

其實只需要編寫`__call__`方法的類，都可以作為優化器，在`train_batch`函數運作時會通過`__call__`方法傳入一個`net_pool`的實例，而優化器只需要回傳一個優化好的池即可。

下面是一個PSO(粒子群演算法)的優化器實作範例:

In [15]:
from nyto import train_tool as train
import random

class PSO:
    def __init__(self, inertia_w=0.99, c1=0.5, c2=0.5, init_v_pool=None):
        self.pbest_pool=None
        self.v_pool=init_v_pool # 初始速度: 如果沒有給定，則在第一次優化時再初始化
        self.w=inertia_w
        self.c1=c1
        self.c2=c2
        
    def __call__(self, pool):
        # 如果是第一次優化，先初始化
        if self.pbest_pool is None: self.pbest_pool=pool.copy()
        if self.v_pool is None: self.v_pool=train.unit_particle(pool)
        
        # 更新經歷過的最好位置
        for idx in range(len(self.pbest_pool)):
            if pool.loss.real[idx]<self.pbest_pool.loss.real[idx]:
                self.pbest_pool.real_net[idx]=pool.net.real[idx]
        
        # 取得全域所經歷最好位置
        gbest_net=self.pbest_pool.net[0]
        
        # 計算各項的速度
        momentum_v=self.w*self.v_pool
        cognition_v=self.c1*random.random()*(self.pbest_pool-pool)
        social_v=self.c2*random.random()*pool.pool_apply(lambda n: gbest_net-n)
        
        # 更新速度
        self.v_pool=momentum_v+cognition_v+social_v
        
        # 更新池中各粒子的位置
        return pool+self.v_pool
        

下面我們就用這個優化器來訓練我們的神經網路，我們請到我們的老朋友鳶尾花資料集作為訓練資料:

In [16]:
from sklearn import datasets
import numpy as np

iris=datasets.load_iris()     # 載入資料
feature=iris.data             # 輸入的特徵資料
label=np.eye(3)[iris.target]  # 預測的類別資料

下面我們建立我們的網路:

In [17]:
from nyto import net_tool as to
from nyto import layer
from nyto import unit_function as uf

(nn, node)=to.new_net(
    layer1=layer.new_nn_layer((4,12)),  # 加入模型節點: layer1(單層神經網路)
    layer2=layer.new_nn_layer((12,3))   # 加入模型節點: layer2(單層神經網路)
)

node.l1_output=node.data_input>>node.layer1>>uf.tanh()
node.pre=node.l1_output>>node.layer2>>uf.softmax()

node.loss=uf.cross_entropy(node.pre, node.data_label)
node.acc=uf.accuracy(node.pre, node.data_label)

然後我們來分割資料並製作批啟動器:

In [18]:
node_list=to.batch_launcher(
    nn=nn,
    get={node.pre, node.loss, node.acc},
    batch_push={'data_input':feature, 'data_label':label},
    batch_size=20
)

在訓練的時候我們只需要loss和acc，所以將其分離出來:

In [19]:
loss_node_list=[n['loss'] for n in node_list]
acc_node_list=[n['acc'] for n in node_list]

最後建立我們的池跟優化器:

In [20]:
pool=train.new_pool(loss_node_list[0], pool_size=20)
opt=PSO()

下面可以開始訓練了，我們打印出每個step的loss和accuracy:

In [21]:
for (t, new_pool, opt) in train.train_batch(loss_node_list, pool, opt):
    (epoch, step)=t
    
    best_loss=new_pool.loss[0]
    best_net=new_pool.net[0]
    
    acc_node_id=acc_node_list[step].node_id
    best_acc=to.get(best_net[acc_node_id])
    
    print(step, best_loss, best_acc)
    
    new_pool # 可以在訓練過程中保存池
    opt      # 也可以在訓練過程中調整優化器的參數
    

0 1.0306472860131364 0.35
1 0.938042095210885 0.9
2 0.9100661461571733 0.6
3 0.790694814382637 0.65
4 0.4552266222837882 0.8
5 0.9441212580689028 0.55
6 1.175445814248707 0.65
7 0.278880016367826 0.85
7 0.278880016367826 0.85


***
*END*